In [1]:
from utilities import *

DATA PRE-PROCESSING

In [2]:
Data = pd.read_csv('Churn.csv')
df =Data.copy()

In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
# dropping columns rownumber, customerid and surname as they will not add value to the models:
df.drop(['CustomerId','Surname'], axis=1, inplace=True)

Binning the Age column to create a categorical variable. 

In [5]:
# Binning the age column
df['Age'].unique()

# Define bins and labels for the age groups
bins = [18, 30, 40, 50, 60, 93]
labels = ['18-29', '30-39', '40-49', '50-59', '60+']

# Bin the ages
age_groups = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

# Display the binned ages
age_groups
#adding this to the df
df = df.assign(Age_binned=age_groups)


Binning the creditscore column to create a categorical variable

In [6]:
# binning the creditscore column
# Define bins for credit score ranges
credit_bins = [349, 450, 550, 650, 750, 850]
credit_labels = ['350-449', '450-549', '550-649', '650-749', '750-850']

# Bin the credit scores
credit_score_groups = pd.cut(df['CreditScore'], bins=credit_bins, labels=credit_labels, right=True)

# Display the binned credit scores
credit_score_groups
#adding this to the df
df = df.assign(CreditScore_binned=credit_score_groups)

In [7]:
# dropping the columns RowNumber, Age and CreditScore
data = df.drop(['RowNumber','Age','CreditScore'], axis=1)

In [8]:
# Loop through the columns and convert only object columns to categorical
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].astype('category')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Geography           10000 non-null  category
 1   Gender              10000 non-null  category
 2   Tenure              10000 non-null  int64   
 3   Balance             10000 non-null  float64 
 4   NumOfProducts       10000 non-null  int64   
 5   HasCrCard           10000 non-null  int64   
 6   IsActiveMember      10000 non-null  int64   
 7   EstimatedSalary     10000 non-null  float64 
 8   Exited              10000 non-null  int64   
 9   Age_binned          10000 non-null  category
 10  CreditScore_binned  10000 non-null  category
dtypes: category(4), float64(2), int64(5)
memory usage: 586.7 KB


TRAIN -VALIDATION-TEST-SPLIT

In [9]:
# sepearting independent from dependent variables
X = data.drop(['Exited'],axis=1)
Y = data['Exited']

# Splitting the dataset into the Training and Test set.
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42,stratify = Y)


# Splitting the Train dataset into the Training and Validation set.
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train, test_size = 0.2, random_state = 42,stratify = y_train)


#Printing the shapes.
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)

(6400, 10) (6400,)
(1600, 10) (1600,)
(2000, 10) (2000,)


Dummy variable creation of categorical cariables

In [10]:
# creating dummy variables
X_train = pd.get_dummies(X_train,drop_first = True)
X_valid = pd.get_dummies(X_valid,drop_first = True)
X_test = pd.get_dummies(X_test,drop_first = True)

print(X_train.shape, X_valid.shape, X_test.shape)
X_train.head()
X_train.columns

(6400, 17) (1600, 17) (2000, 17)


Index(['Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Geography_Germany', 'Geography_Spain',
       'Gender_Male', 'Age_binned_30-39', 'Age_binned_40-49',
       'Age_binned_50-59', 'Age_binned_60+', 'CreditScore_binned_450-549',
       'CreditScore_binned_550-649', 'CreditScore_binned_650-749',
       'CreditScore_binned_750-850'],
      dtype='object')

Data normalization 

In [11]:
#Standardizing the numerical variables to zero mean and unit variance.
transformer = StandardScaler()
X_train[['Tenure','Balance','NumOfProducts','EstimatedSalary']] = transformer.fit_transform(X_train[['Tenure','Balance','NumOfProducts','EstimatedSalary']])
X_valid[['Tenure','Balance','NumOfProducts','EstimatedSalary']] = transformer.fit_transform(X_valid[['Tenure','Balance','NumOfProducts','EstimatedSalary']])
X_test[['Tenure','Balance','NumOfProducts','EstimatedSalary']] = transformer.fit_transform(X_test[['Tenure','Balance','NumOfProducts','EstimatedSalary']])

Building base model